In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')

In [50]:
# Verificación inicial
print(f"Sell-In: {sellin.shape[0]} filas y {sellin.shape[1]} columnas")
print(f"Productos: {productos.shape[0]} filas y {productos.shape[1]} columnas")
print(f"Stocks: {stocks.shape[0]} filas y {stocks.shape[1]} columnas")

Sell-In: 2945818 filas y 7 columnas
Productos: 1262 filas y 6 columnas
Stocks: 13691 filas y 3 columnas


In [51]:
# 3. MERGE INICIAL
df = sellin.merge(productos, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")

Ventas-Productos-Stocks: 2988650 filas y 13 columnas


In [52]:
productos_clean = productos.drop_duplicates(subset=['product_id'], keep='first')
print(productos_clean.shape)

(1251, 6)


In [53]:
df = sellin.merge(productos_clean, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(sellin.shape)
print(df.shape)

(2945818, 7)
(2945818, 13)


In [54]:
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

In [83]:
# Supongamos que df ya contiene las columnas: periodo, customer_id, product_id, tn
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

# Paso 1: Rango total de periodos
todos_los_periodos = pd.date_range(start=df["periodo_dt"].min(), end=df["periodo_dt"].max(), freq="MS")

# Paso 2: Todos los clientes únicos
todos_los_clientes = df["customer_id"].unique()

# Paso 3: Determinar vida útil de cada producto
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()

# Paso 4: Generar combinaciones (periodo, producto) considerando restricciones
combinaciones_producto_periodo = []
fecha_limite_nuevos = pd.to_datetime("2017-03", format="%Y-%m")

# Los productos de 35 y 36 meeses de vida son considerados "vitales"
productos_vitales = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()
mask = (productos_vitales['min'] == '2017-01-01') & ((productos_vitales['max'] == '2019-12-01'))
productos_vitales = productos_vitales[mask]['product_id'].unique()  

for _, row in vida_producto.iterrows():
    producto = row["product_id"]
    min_fecha = row["min"]
    max_fecha = row["max"]
    periodos_validos = pd.date_range(start=min_fecha, end=max_fecha, freq="MS")
    es_nuevo = min_fecha >= fecha_limite_nuevos  # solo si el producto es nuevo a partir de 2017-02
    
    for p in periodos_validos:
        if producto in productos_vitales:
            combinaciones_producto_periodo.append((p, producto))
            continue
        # Excluir primeros 3 meses si es nuevo (a partir de 2017-02)
        if es_nuevo and (p < min_fecha + pd.DateOffset(months=3)):
            continue
        # Excluir últimos 3 meses del producto
        if p > max_fecha - pd.DateOffset(months=3):
            continue
        combinaciones_producto_periodo.append((p, producto))

df_producto_periodo = pd.DataFrame(combinaciones_producto_periodo, columns=["periodo_dt", "product_id"])

# Paso 5: Generar combinaciones de todos los clientes con (periodo, producto)
combinaciones = []
for _, row in df_producto_periodo.iterrows():
    periodo = row["periodo_dt"]
    producto = row["product_id"]
    for cliente in todos_los_clientes:
        # if producto in df[df["customer_id"] == cliente]["product_id"].unique(): ###### <------ ESTO TARDA 3 AÑOS
        combinaciones.append((periodo, producto, cliente)) 

df_completo = pd.DataFrame(combinaciones, columns=["periodo_dt", "product_id", "customer_id"])

# Paso 6: Unir con toneladas efectivas
df_merge = df_completo.merge(df[["periodo_dt", "product_id", "customer_id", "tn"]],
                             on=["periodo_dt", "product_id", "customer_id"],
                             how="left")
df_merge["tn"] = df_merge["tn"].fillna(0)

# Paso 7: Recuperar periodo AAAAMM si lo necesitás
df_merge["periodo"] = df_merge["periodo_dt"].dt.strftime("%Y%m").astype(int)

# Resultado final
df_final = df_merge[["periodo", "product_id", "customer_id", "tn"]]

# Vista previa
print(df_final.head())

   periodo  product_id  customer_id        tn
0   201701       20001        10234   0.33579
1   201701       20001        10032  12.31230
2   201701       20001        10217   0.00000
3   201701       20001        10125   0.08954
4   201701       20001        10012   6.97324


In [84]:
a = df_final[df_final['periodo']==201912]
a['product_id'].nunique()

927

In [85]:
df_final.shape

(18818634, 4)

In [ ]:
# # 📅 Convertir periodos a datetime
# sellin['periodo_dt'] = pd.to_datetime(sellin['periodo'].astype(str), format='%Y%m')
# stocks['periodo_dt'] = pd.to_datetime(stocks['periodo'].astype(str), format='%Y%m')

# # 🛠 Preparar vida útil productos desde sellin
# vida_producto = sellin.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()
# fecha_limite_nuevos = pd.to_datetime("2017-03", format="%Y-%m")

# # Identificar productos vitales (35-36 meses)
# productos_vitales = vida_producto[
#     (vida_producto['min'] == pd.to_datetime('2017-01-01')) &
#     (vida_producto['max'] == pd.to_datetime('2019-12-01'))
# ]['product_id'].unique()

# # Generar combinaciones periodo-producto según reglas
# combinaciones_producto_periodo = []
# for _, row in vida_producto.iterrows():
#     producto = row["product_id"]
#     min_fecha = row["min"]
#     max_fecha = row["max"]
#     periodos_validos = pd.date_range(start=min_fecha, end=max_fecha, freq="MS")
#     es_nuevo = min_fecha >= fecha_limite_nuevos
    
#     for p in periodos_validos:
#         if producto in productos_vitales:
#             combinaciones_producto_periodo.append((p, producto))
#             continue
#         if es_nuevo and (p < min_fecha + pd.DateOffset(months=3)):
#             continue
#         if p > max_fecha - pd.DateOffset(months=3):
#             continue
#         combinaciones_producto_periodo.append((p, producto))

# df_producto_periodo = pd.DataFrame(combinaciones_producto_periodo, columns=["periodo_dt", "product_id"])

# # Obtener todos los clientes
# todos_clientes = sellin['customer_id'].unique()

# # 🚀 Generar combinaciones periodo-producto-cliente
# combinaciones = []
# for _, row in df_producto_periodo.iterrows():
#     for cliente in todos_clientes:
#         combinaciones.append((row['periodo_dt'], row['product_id'], cliente))

# df_completo = pd.DataFrame(combinaciones, columns=["periodo_dt", "product_id", "customer_id"])

# # 🚦 Cruce con ventas (sell-in)
# sellin_subset = sellin[['periodo_dt', 'product_id', 'customer_id', 'tn']]
# df_merged = df_completo.merge(sellin_subset, on=['periodo_dt', 'product_id', 'customer_id'], how='left')
# df_merged['tn'] = df_merged['tn'].fillna(0)

# # 🚦 Cruce con tb_productos (por product_id)
# df_merged = df_merged.merge(productos, on='product_id', how='left')

# # 🚦 Cruce con stocks (por periodo y product_id)
# df_merged = df_merged.merge(stocks[['periodo_dt', 'product_id', 'stock_final']], on=['periodo_dt', 'product_id'], how='left')

# # 📅 Recuperar periodo en formato AAAAMM
# df_merged['periodo'] = df_merged['periodo_dt'].dt.strftime('%Y%m').astype(int)

# # 🚀 Dataset final para LightGBM
# print(df_merged.head())


  periodo_dt  product_id  customer_id        tn cat1         cat2     cat3  \
0 2017-01-01       20001        10234   0.33579   HC  ROPA LAVADO  Liquido   
1 2017-01-01       20001        10032  12.31230   HC  ROPA LAVADO  Liquido   
2 2017-01-01       20001        10217   0.00000   HC  ROPA LAVADO  Liquido   
3 2017-01-01       20001        10125   0.08954   HC  ROPA LAVADO  Liquido   
4 2017-01-01       20001        10012   6.97324   HC  ROPA LAVADO  Liquido   

   brand  sku_size  stock_final  periodo  
0  ARIEL    3000.0          NaN   201701  
1  ARIEL    3000.0          NaN   201701  
2  ARIEL    3000.0          NaN   201701  
3  ARIEL    3000.0          NaN   201701  
4  ARIEL    3000.0          NaN   201701  


In [ ]:
df_merged.shape

(17193600, 11)

In [77]:
df_merged['product_id'].nunique()

1110

In [87]:
# Contar duplicados en todo el DataFrame
num_duplicados = df_final.duplicated().sum()
print(f"Cantidad total de filas duplicadas en df_final: {num_duplicados}")


Cantidad total de filas duplicadas en df_final: 0


In [ ]:
# ts = df_merged.drop_duplicates()
# ts.shape


(17000172, 11)

In [89]:
ts = df_final.copy()
ts['periodo_dt'] = pd.to_datetime(ts['periodo'].astype(str), format='%Y%m')
# crear alguans variables nuevas de prueba
ts["month"] = ts["periodo_dt"].dt.month
ts["year"] = ts["periodo_dt"].dt.year


# Convertir columnas categóricas a tipo 'category' para LightGBM
for col in ['cat1', 'cat2', 'cat3', 'brand']:
    if col in ts.columns:
        ts[col] = ts[col].astype('category')

In [90]:
# Convertir 'periodo_dt' a formato de tiempo si no está
ts['periodo_dt'] = pd.to_datetime(ts['periodo_dt'])

# Asegurar que está ordenado por producto y fecha
ts = ts.sort_values(['product_id', 'periodo_dt'])

# Crear lags de ventas (tn)
for lag in range(1, 12):  # Por ejemplo, lags 1, 2 y 3 meses
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

# Crear delta (diferencia) con el mes anterior
ts['tn_delta'] = ts['tn'] - ts['tn_lag_1']

# Crear delta relativo (proporcional)
ts['tn_delta_pct'] = ts['tn_delta'] / ts['tn_lag_1'].replace(0, np.nan)

# Crear rolling stats (medias y desviaciones)
for window in [3, 6, 12, 24]:  # Por ejemplo, ventanas de 3 y 6 meses
    ts[f'tn_roll_mean_{window}'] = ts.groupby('product_id')['tn'].rolling(window).mean().reset_index(0, drop=True)
    ts[f'tn_roll_std_{window}'] = ts.groupby('product_id')['tn'].rolling(window).std().reset_index(0, drop=True)

# Crear características de estacionalidad
ts['month'] = ts['periodo_dt'].dt.month
ts['quarter'] = ts['periodo_dt'].dt.quarter
ts['year'] = ts['periodo_dt'].dt.year
ts['season'] = ts['month'].apply(lambda x: 1 if x in [6, 7, 8] else 0)  # ejemplo: verano

# Si querés completar valores nulos generados por lags y rolling
# ts.fillna(0, inplace=True)

print(ts.head())



   periodo  product_id  customer_id        tn periodo_dt  month  year  \
0   201701       20001        10234   0.33579 2017-01-01      1  2017   
1   201701       20001        10032  12.31230 2017-01-01      1  2017   
2   201701       20001        10217   0.00000 2017-01-01      1  2017   
3   201701       20001        10125   0.08954 2017-01-01      1  2017   
4   201701       20001        10012   6.97324 2017-01-01      1  2017   

   tn_lag_1  tn_lag_2  tn_lag_3  ...  tn_roll_mean_3  tn_roll_std_3  \
0       NaN       NaN       NaN  ...             NaN            NaN   
1   0.33579       NaN       NaN  ...             NaN            NaN   
2  12.31230   0.33579       NaN  ...        4.216030       7.013585   
3   0.00000  12.31230   0.33579  ...        4.133947       7.082803   
4   0.08954   0.00000  12.31230  ...        2.354260       4.000405   

   tn_roll_mean_6  tn_roll_std_6  tn_roll_mean_12  tn_roll_std_12  \
0             NaN            NaN              NaN             NaN

In [91]:
ts['tn_target'] = ts['tn'].shift(-2)

In [92]:
dt_kgl = ts[ts["periodo"].isin([201912])]
ts = ts.drop(ts[ts["periodo"].isin([201911,201912])].index,axis=0)

In [93]:
dt_kgl['product_id'].nunique()

927

# Sin optuna

In [ ]:
import lightgbm as lgb

feature_columns = [col for col in ts.columns if col not in ['periodo_dt', 'tn_target']]

X = ts[feature_columns]
y = ts['tn_target']

lgb_reg = lgb.LGBMRegressor(random_state=12345)

# Entrenar el modelo
lgb_reg.fit(X, y)

print("Modelo LightGBM entrenado con éxito.")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.070821 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 799
[LightGBM] [Info] Number of data points in the train set: 17702244, number of used features: 4
[LightGBM] [Info] Start training from score 0.071443
Modelo LightGBM entrenado con éxito.


# Con optuna

In [114]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np

# Preparar datos
feature_columns = [col for col in ts.columns if col not in ['periodo_dt', 'tn_target']]
feature_columns = ['periodo', 'customer_id','product_id','tn']
X = ts[feature_columns]
y = ts['tn_target']

# Verificar NaN en y y eliminarlos
if y.isnull().any():
    print("⚠️ Target tiene NaN, se eliminarán.")
    mask = ~y.isnull()
    X = X[mask]
    y = y[mask]

# Opcional: usar una muestra para pruebas (descomentar si querés)
# X = X.sample(frac=0.2, random_state=42)
# y = y.loc[X.index]

# Función objetivo para Optuna
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'random_state': 12345,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)
    }

    tscv = TimeSeriesSplit(n_splits=3)  # Ajustar según disponibilidad de memoria
    rmses = []
    for train_idx, valid_idx in tscv.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[
                lgb.early_stopping(50),
                lgb.log_evaluation(0)  # Desactiva logs
            ]
        )

        preds = model.predict(X_valid)
        rmse = mean_squared_error(y_valid, preds, squared=False)
        rmses.append(rmse)

    return np.mean(rmses)

# Crear y optimizar estudio Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Mostrar mejores parámetros encontrados
print("Mejores parámetros encontrados:", study.best_params)

# Entrenar modelo final con los mejores parámetros
best_params = study.best_params
best_model = lgb.LGBMRegressor(**best_params, random_state=12345)
best_model.fit(X, y)

print("✅ Modelo LightGBM optimizado y entrenado con éxito.")


⚠️ Target tiene NaN, se eliminarán.


MemoryError: Unable to allocate 16.9 MiB for an array with shape (17702242,) and data type bool

In [111]:
print("¿X tiene NaN?:", X.isnull().any().any())
print("¿y tiene NaN?:", y.isnull().any())


¿X tiene NaN?: True
¿y tiene NaN?: True


# Prediccion

In [102]:
X_kgl = dt_kgl[feature_columns]

productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")
# Filtrar filas
productos_filtrados = productos_a_predecir['product_id'].unique()
X_kgl = X_kgl[X_kgl['product_id'].isin(productos_filtrados)]

X_kgl['product_id'].nunique()

780

In [104]:
y_pred = lgb_reg.predict(X_kgl)

In [105]:
result = pd.DataFrame({"product_id": X_kgl["product_id"],  "tn": y_pred})
result = result.groupby("product_id").agg({"tn":"sum"}).reset_index()
result

,product_id,tn
0,20001,1116.043838
1,20002,1122.006487
2,20003,1080.209452
3,20004,1089.665181
4,20005,557.805169
...,...,...
775,21263,2.179084
776,21265,2.181989
777,21266,2.181989
778,21267,2.179177


In [106]:
result.to_csv("./kaggle/05-lgb_customer_x_cliente.csv", index=False, sep=",")